In [8]:
import torch
from torch import nn
from torch.utils.data import DataLoader
import torch.optim as optim
from sklearn.model_selection import train_test_split
import pandas as pd

In [9]:
'''
    CODE TO PREPARE DATA.
'''
train_path = 'dataset/train.csv'
test_path = 'dataset/test.csv'

dataset = pd.read_csv(train_path, header=0)
dataset.astype(float)
submission_dataset = pd.read_csv(test_path, header=0)

y = dataset['label']
X = dataset.drop(labels='label', axis=1)


In [10]:
x_train, x_test, y_train, y_test = train_test_split(
  X,y , random_state=104,test_size=0.25, shuffle=True)

# train_data = x_train
# train_data['label'] = y_train

# test_data = x_test
# test_data['label'] = y_test

# train_dataset = torch.tensor(train_data.values)
# test_dataset = torch.tensor(test_data.values)

x_train = torch.tensor(x_train.values)
y_train = torch.tensor(y_train.values)
x_test = torch.tensor(x_test.values)
y_test = torch.tensor(y_test.values)

training_set = torch.utils.data.TensorDataset(x_train, y_train)
validation_set = torch.utils.data.TensorDataset(x_test, y_test)

trainloader = torch.utils.data.DataLoader(training_set,
                                          batch_size=10,
                                          shuffle=False, num_workers=2)
validationloader = torch.utils.data.DataLoader(validation_set,
                                         shuffle=False, num_workers=2)



In [11]:
import torch.nn.functional as F
class net(nn.Module):
    def __init__(self):
        super(net, self).__init__()
        self.l1 = nn.Linear(784, 500)
        self.l2 = nn.Linear(500, 250)
        self.l3 = nn.Linear(250, 10)
    def forward(self, x):
        x = self.l1(x.to(torch.float))
        x = F.relu(x.to(torch.float))  # Convert to float before applying ReLU
        x = self.l2(x)
        x = F.relu(x.to(torch.float))  # Convert to float before applying ReLU
        x = self.l3(x)
        return x

model = net()
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001)
training_loss = []

In [12]:
print(trainloader)
for i, data in enumerate(trainloader):
    print(data)
    break

[tensor([[0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        ...,
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0]]), tensor([3, 9, 4, 5, 2, 7, 4, 3, 5, 9])]


In [13]:
training_loss = []
from tqdm import tqdm
for epoch in tqdm(range(200)):
    running_loss = 0
    for i, data in enumerate(trainloader):
        x, y = data
        optimizer.zero_grad()
        output = model(x)
        _, y_hat = torch.max(output, 1)  # Use torch.max directly for class indices
        tloss = criterion(output, y)
        tloss.backward()
        optimizer.step()
        running_loss += tloss.item()
    print(running_loss)
    training_loss.append(running_loss)

  0%|          | 1/200 [00:23<1:18:44, 23.74s/it]

1029.4462064928084


In [ ]:
from matplotlib import pyplot as plt
plt.figure()
plt.plot(training_loss)